In [1]:
import numpy as np
import pandas as pd
import glob
import os
import tqdm

In [2]:
def cm_to_iou(cm):
    tps = np.diag(cm)
    fps = cm.sum(axis=0) - tps
    fns = cm.sum(axis=1) - tps
    iou = tps / (tps + fps + fns)
    return iou.squeeze()

def cm_to_miou(cm):
    iou = cm_to_iou(cm)
    return iou.mean()

def read_metrics(metrics_path):
    with open(metrics_path, 'r') as f:
        lines = f.readlines()
        miou = float(lines[-1].split(':')[-1].strip())
    return miou

def add2dict(metrics_dict, label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory, cm, miou):
    metrics_dict['dataset'].append(dataset)
    metrics_dict['trajectory'].append(trajectory)
    metrics_dict['label_set'].append(label_set)
    metrics_dict['seg_method'].append(seg_method)
    metrics_dict['lulc_type'].append(lulc_type)
    metrics_dict['d3_type'].append(d3_type)
    metrics_dict['res'].append(res)
    metrics_dict['refine_type'].append(refine_type)
    metrics_dict['cm'].append(cm)
    # metrics_dict['iou'].append(iou)
    metrics_dict['miou'].append(miou)

In [3]:
common_idx2names = {
    0 : 'water',
    1 : 'trees',
    2 : 'low_vegetation',
    3 : 'built',
    4 : 'ground',
    5 : 'sky',
}

more_common_idx2names = {
    0 : 'water',
    1 : 'vegetation',
    2 : 'built',
    3 : 'ground',
    4 : 'sky',
}

most_common_idx2names = {
    0 : 'water',
    1 : 'ground',
    2 : 'sky',
}

In [4]:
cm_paths = glob.glob('./classical_outputs/*/*/*/*/*/*/*/*/confusion_matrix.npy')

In [5]:
metrics_dict = {
    'dataset' : [],
    'trajectory' : [],
    'label_set' : [],
    'seg_method' : [],
    'lulc_type' : [],
    'd3_type' : [],
    'res' : [],
    'refine_type' : [],
    'cm' : [],
    # 'iou' : [],
    'miou' : [],
}

for cm_filepath in tqdm.tqdm(cm_paths):
    iou_path = cm_filepath.replace('confusion_matrix.npy', 'metrics.txt')
    miou = read_metrics(iou_path)

    label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory = cm_filepath.split(os.path.sep)[-9:-1]
    cm = np.load(cm_filepath)
    iou = cm_to_iou(cm)
    
    add2dict(metrics_dict, label_set, seg_method, lulc_type, d3_type, res, refine_type, dataset, trajectory, cm, miou)
    
    # Colorado has no DSM. Use dem 1m instead.
    # Lets us compare dem vs dsm
    if dataset == '2022-05-15_ColoradoRiver' and d3_type == 'dem':
        add2dict(metrics_dict, label_set, seg_method, lulc_type, 'dsm', res, refine_type, dataset, trajectory, cm, miou)

    # Lets us compare dem vs dem_1m
    if dataset == '2023-03-XX_Duck' and d3_type == 'dem':
        add2dict(metrics_dict, label_set, seg_method, lulc_type, 'dem_1m', res, refine_type, dataset, trajectory, cm, miou)

    


  0%|          | 0/156 [00:00<?, ?it/s]/tmp/ipykernel_3536244/3340967550.py:5: RuntimeWarning: invalid value encountered in divide
  iou = tps / (tps + fps + fns)
100%|██████████| 156/156 [00:00<00:00, 5811.50it/s]


In [6]:
df = pd.DataFrame.from_dict(metrics_dict)

In [7]:
unique_ids = ['label_set', 'seg_method', 'lulc_type', 'd3_type', 'res', 'refine_type']
grouped_df = df.groupby(unique_ids)

In [8]:
counts = grouped_df['miou'].transform('count')
cm_sum = grouped_df['cm'].transform('sum')
trajectory_avg_miou = grouped_df['miou'].transform('mean')

df['trajectory_count'] = counts
df['cm_sum'] = cm_sum
df['trajectory_avg_miou'] = trajectory_avg_miou
df['miou (total)'] = df.apply(lambda x: cm_to_miou(x['cm_sum']), axis=1)


In [9]:
# df_agg = df.groupby(['label_set', 'seg_method', 'lulc_type', 'd3_type', 'res', 'refine_type']).reset_index()
df_agg_ds_stats = df.drop(columns=['dataset', 'trajectory', 'cm', 'cm_sum', 'miou'])
df_agg_ds_stats

,label_set,seg_method,lulc_type,d3_type,res,refine_type,trajectory_count,trajectory_avg_miou,miou (total)
0,most_common,felzenszwalb,dynamicworld,dsm,1.0,none,11,0.668763,0.761468
1,most_common,felzenszwalb,dynamicworld,dsm,1.0,none,11,0.668763,0.761468
2,most_common,felzenszwalb,dynamicworld,dsm,1.0,none,11,0.668763,0.761468
3,most_common,felzenszwalb,dynamicworld,dsm,1.0,none,11,0.668763,0.761468
4,most_common,felzenszwalb,dynamicworld,dsm,1.0,none,11,0.668763,0.761468
...,...,...,...,...,...,...,...,...,...
193,common,slic,dynamicworld,dem_1m,1.0,none,11,0.354515,0.516166
194,common,slic,dynamicworld,dem_1m,1.0,none,11,0.354515,0.516166
195,common,slic,dynamicworld,dem_1m,1.0,none,11,0.354515,0.516166
196,common,slic,dynamicworld,dem_1m,1.0,none,11,0.354515,0.516166


In [10]:
df_agg_ds_stats.drop_duplicates(inplace=True)

In [46]:
seg_method = 'felzenszwalb'
filter_cond = (
    (df_agg_ds_stats['seg_method'] == seg_method) &
    (df_agg_ds_stats['d3_type'] == 'dem') &
    (df_agg_ds_stats['res'] == '1.0')
)

print(df_agg_ds_stats[filter_cond][['trajectory_avg_miou', 'miou (total)']].mean())
print(df_agg_ds_stats[filter_cond][['label_set', 'trajectory_avg_miou', 'miou (total)']].set_index('label_set').T[['common', 'more_common', 'most_common']].to_latex(float_format='{:.3f}'.format))

trajectory_avg_miou    0.479923
miou (total)           0.596333
dtype: float64
\begin{tabular}{lrrr}
\toprule
label_set & common & more_common & most_common \\
\midrule
trajectory_avg_miou & 0.345 & 0.417 & 0.677 \\
miou (total) & 0.495 & 0.521 & 0.773 \\
\bottomrule
\end{tabular}



In [45]:
seg_method = 'slic'
filter_cond = (
    (df_agg_ds_stats['seg_method'] == seg_method) &
    (df_agg_ds_stats['d3_type'] == 'dem') &
    (df_agg_ds_stats['res'] == '1.0')
)

print(df_agg_ds_stats[filter_cond][['trajectory_avg_miou', 'miou (total)']].mean())
print(df_agg_ds_stats[filter_cond][['label_set', 'trajectory_avg_miou', 'miou (total)']].set_index('label_set').T[['common', 'more_common', 'most_common']].to_latex(float_format='{:.3f}'.format))

trajectory_avg_miou    0.506432
miou (total)           0.623683
dtype: float64
\begin{tabular}{lrrr}
\toprule
label_set & common & more_common & most_common \\
\midrule
trajectory_avg_miou & 0.365 & 0.443 & 0.711 \\
miou (total) & 0.526 & 0.545 & 0.801 \\
\bottomrule
\end{tabular}

